# Segmentation Notebook

In [ ]:
# from organoid import segmentation
from organoid import preprocessing
from organoid import reconstruction
from organoid import analysis
from organoid import utils
import numpy as np
import tifffile
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
from skimage.measure import regionprops

In [ ]:
path = rf"C:\Users\gros\Desktop\DATA\organoid_test"
data = tifffile.imread(rf"{path}\data.tif")
dapi = data[:, 0, :, :]
stardist_model = rf"C:\Users\gros\Desktop\CODES\Alice_Segmentation\Stardist\models\lennedist_3d_grid222_rays64"

In [ ]:
output = segmentation.predict_stardist(
    data,
    model_path=stardist_model,
    input_voxelsize=[1, 1, 1],
    normalize_input=True,
)

Threshold and normalize your division channel to run the prediction

In [ ]:

path_mask = ...
path_array = ...
path_output= ...
num= 1

mask = tifffile.imread(path_mask)
image = tifffile.imread(path_array)
division_channel = ...
dapi = ...

ph3=utils.normalize_by_reference_signal(signal = division_channel,reference=dapi,mask=mask)

blurred_data=ndi.gaussian_filter(ph3,sigma=1.2)

threshold = 0.8
blurred_data[blurred_data<threshold]=0

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,5))
z =int(len(blurred_data)/2)
ax1.imshow(ph3[z,:,:])
ax2.imshow(blurred_data[z,:,:])
plt.tight_layout()

In [ ]:
stardist_model_divisions = ...
labels_division = segmentation.predict_stardist(blurred_data, model_path=stardist_model_divisions, input_voxelsize=[1, 1, 1], normalize_input=True)
print('Number of labels found :', len(np.unique(labels_division)))
tifffile.imsave(path_output,labels_division)

In [ ]:
#if napari installed
# import napari
# viewer=napari.Viewer()
# viewer.add_image(ph3,colormap='inferno')
# viewer.add_image(blurred_data,colormap='inferno')
# napari.run()

Apply mask to remove volumes that are out of the samples

In [ ]:
path_mask = ...
mask = tifffile.imread(path_mask)
labels_division=labels_division*mask.astype(bool)

Filter wrong volumes in the segmentation

In [ ]:
props=regionprops(labels_division)
histo=[]
for prop in props :
    bb = prop.slice #bounding box around the cell
    cell = prop.image #boolean array giving where the cell is
    histo.append(np.sum(cell))


plt.hist(histo,bins=100)
plt.title('Histogram of cell volumes')

In [ ]:
#choose the size to filter below
size_min =1000

labels_division_filtered = labels_division.copy()
unique_labels, label_counts = np.unique(labels_division, return_counts=True)

smallest_labels = unique_labels[np.argsort(label_counts)]
smallest_volumes = np.sort(label_counts)

for label, volume in zip(smallest_labels, smallest_volumes):
    if volume<size_min :
        labels_division_filtered[labels_division==label]=0

print('The prediction returned',len(np.unique(labels_division)),'labels, after filtering we have now',len(np.unique(labels_division_filtered)),'labels')

In [ ]:
# viewer=napari.Viewer()
# viewer.add_image(ph3,scale=scale)
# viewer.add_labels(labels_division_filtered,scale=scale,name='labels_filtered')
# viewer.add_labels(labels_division,scale=scale,name='labels_not_filtered')
# napari.run()